In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import glob
import os
%matplotlib inline

In [2]:
path = "/mnt/storage/home/ja17618/scratch/DATA_DIR/training/"
f_lidar = glob.glob(os.path.join(path, 'velodyne', '*.bin'))
f_rgb = glob.glob(os.path.join(path, 'image_2', '*.png'))
f_labels = glob.glob(os.path.join(path, 'label_2', '*.txt'))
data_tag = [name.split('/')[-1].split('.')[-2] for name in f_lidar]
files=[tag+".bin" for tag in data_tag]

In [7]:
colors = {
    'Car': 'b',
    'Tram': 'r',
    'Cyclist': 'g',
    'Van': 'c',
    'Truck': 'm',
    'Pedestrian': 'y',
    'Sitter': 'k',
    'DontCare' : 'k'
}
axes_limits = [
    [-20, 80], # X axis range
    [-20, 20], # Y axis range
    [-3, 10]   # Z axis range
]
axes_str = ['X', 'Y', 'Z']

def draw_box(pyplot_axis, vertices, axes=[0, 1, 2], color='black'):
    """
    Draws a bounding 3D box in a pyplot axis.
    
    Parameters
    ----------
    pyplot_axis : Pyplot axis to draw in.
    vertices    : Array 8 box vertices containing x, y, z coordinates.
    axes        : Axes to use. Defaults to `[0, 1, 2]`, e.g. x, y and z axes.
    color       : Drawing color. Defaults to `black`.
    """
    vertices = vertices[axes, :]
    connections = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Lower plane parallel to Z=0 plane
        [4, 5], [5, 6], [6, 7], [7, 4],  # Upper plane parallel to Z=0 plane
        [0, 4], [1, 5], [2, 6], [3, 7]  # Connections between upper and lower planes
    ]
    for connection in connections:
        pyplot_axis.plot(*vertices[:, connection], c=color, lw=0.5)

def display_frame_statistics(dataset_velo, dataset_rgb, dataset_label, points=1.0):
    """
    Displays statistics for a single frame. Draws camera data, 3D plot of the lidar point cloud data and point cloud
    projections to various planes.
    
    Parameters
    ----------
    dataset         : `raw` dataset.

    points          : Fraction of lidar points to use. Defaults to `0.2`, e.g. 20%.
    """
# #     dataset_gray = list(dataset.gray)
#     dataset_rgb =  dataset+".png"
#     dataset_velo = dataset+".bin"
    
#     print('Frame timestamp: ' + str(dataset.timestamps[frame]))
    # Draw camera data
    box_3d=[]
    cls=[]
    for line in dataset_label: 
        ret = line.split()
        cls.append(ret[0])
        h, w, l, x, y, z, r = [float(i) for i in ret[-7:]]
        box3d = np.array([h, w, l, x, y, z, r])
        box_3d.append(box3d)
    
    f, ax = plt.subplots(1, 1, figsize=(15, 5))

    ax.imshow(dataset_rgb)
    ax.set_title('Right RGB Image (cam3)')
#     plt.show()
    
    
        
    points_step = int(1. / points)
    point_size = 0.1 * (1. / points)
    velo_range = range(0, dataset_velo.shape[0], points_step)
    velo_frame = dataset_velo[velo_range, :]      
    def draw_point_cloud(ax, title, axes=[0, 1, 2], xlim3d=None, ylim3d=None, zlim3d=None):
        """
        Convenient method for drawing various point cloud projections as a part of frame statistics.
        """
        ax.set_facecolor('xkcd:black')
        ax.scatter(*np.transpose(velo_frame[:, axes]), s=point_size, c=velo_frame[:, 3], cmap='jet')
        ax.set_title(title)
        ax.set_xlabel('{} axis'.format(axes_str[axes[0]]))
        ax.set_ylabel('{} axis'.format(axes_str[axes[1]]))
        if len(axes) > 2:
            ax.set_xlim3d(*axes_limits[axes[0]])
            ax.set_ylim3d(*axes_limits[axes[1]])
            ax.set_zlim3d(*axes_limits[axes[2]])
            ax.set_zlabel('{} axis'.format(axes_str[axes[2]]))
        else:
            ax.set_xlim(*axes_limits[axes[0]])
            ax.set_ylim(*axes_limits[axes[1]])
        # User specified limits
        if xlim3d!=None:
            ax.set_xlim3d(xlim3d)
        if ylim3d!=None:
            ax.set_ylim3d(ylim3d)
        if zlim3d!=None:
            ax.set_zlim3d(zlim3d)
            
#         for t_rects, t_type in zip(box_3d, cls):
#             print(t_rects,t_type)
#             draw_box÷(ax, t_rects, axes=axes, color=colors[t_type])
            
    # Draw point cloud data as 3D plot
    f2 = plt.figure(figsize=(15, 8))
    ax2 = f2.add_subplot(111, projection='3d')    
    max_x = np.max(dataset_velo[:,0])
    draw_point_cloud(ax2, 'Velodyne scan', xlim3d=(-10,max_x),ylim3d=(-max_x/2,max_x/2))
    plt.show()
    
    # Draw point cloud data as plane projections
    f, ax3 = plt.subplots(3, 1, figsize=(15, 25))
    draw_point_cloud(
        ax3[0], 
        'Velodyne scan, XZ projection (Y = 0), the car is moving in direction left to right', 
        axes=[0, 2] # X and Z axes
    )
    draw_point_cloud(
        ax3[1], 
        'Velodyne scan, XY projection (Z = 0), the car is moving in direction left to right', 
        axes=[0, 1] # X and Y axes
    )
    draw_point_cloud(
        ax3[2], 
        'Velodyne scan, YZ projection (X = 0), the car is moving towards the graph plane', 
        axes=[1, 2] # Y and Z axes
    )
    plt.show()

In [ ]:
for image,pcl,label in zip(f_rgb,f_lidar,f_labels):
    print("File Number %s" %(imag
                             e.split('/')[-1].split('.')[-2]))
    raw_lidar = np.fromfile(pcl,dtype=np.float32).reshape((-1,4))
    camera = cv2.imread(image,)
    camera = cv2.cvtColor(camera, cv2.COLOR_BGR2RGB)
    labels= [line for line in open(label,'r').readlines()]
    display_frame_statistics(raw_lidar,camera,labels,points=1.0)
    input()
